In [2]:
import pygame as pg
import numpy as np
import pdb


In [3]:
def loadIcons():
    possible_icons  = [1,2,3,6,12,24,48,96,192,384,768,1536,3072,6144, 12288]
    icons = dict()
    for index in possible_icons:
        icons[index] =pg.image.load('C:/Henry/PythonProjects/Pygame/Threes/images/_' + str(index) + '.png')
    return icons 

In [120]:
class Tiles:
    def __init__(self, tile_count):
        
        inital_number = np.random.randint(1,3,(tile_count,))
        self.grid_pos = -1*np.ones((tile_count,),dtype = int) 
        self.number = np.zeros((tile_count,),dtype = int)  
        initial_grid = np.random.permutation(16)
        self.icons =  loadIcons()
        for index in range(tile_count):
            self.grid_pos[index] = initial_grid[index]
            self.number[index] = inital_number[index]
            self.drawTile(index)
        
       
    
    def determineMove(self,keys):
        current_move = [0, 0]
        move_order = np.argsort(self.grid_pos)
        if keys[pg.K_LEFT]:
            current_move = [0, -1]
            move_order = np.argsort(self.grid_pos)
                    
        elif keys[pg.K_UP]:
            current_move = [-1, 0]
            move_order = np.argsort(self.grid_pos)
    
        elif keys[pg.K_RIGHT]:
            current_move = [0, 1]
            move_order = np.argsort(self.grid_pos)
            move_order = move_order[-1::-1]
        elif keys[pg.K_DOWN]:
            current_move = [1,0]
            move_order = np.argsort(self.grid_pos)
            move_order = move_order[-1::-1]
            
        return current_move, move_order
    def updatePosition(self,current_move,move_order):
        delete_vector = np.zeros(len(move_order),dtype = bool)
        for index  in move_order:
            if np.sum(current_move) != 0 and self.grid_pos[index] > -1:
                if current_move[1] == -1  and self.grid_pos[index]>3: # move left
                    
                    new_pos = self.grid_pos[index]-4
                    
                    self, delete_vector[index] = self.checkMovement(index,new_pos)
                    
                if current_move[1] == 1  and self.grid_pos[index]<12: # move right
                    new_pos = self.grid_pos[index]+4
                    self, delete_vector[index] = self.checkMovement(index,new_pos)
                        
                if current_move[0] == -1  and np.mod(self.grid_pos[index],4)>0: # move up
                    
                    new_pos = self.grid_pos[index]-1
                    self, delete_vector[index] = self.checkMovement(index,new_pos)
                    
                if current_move[0] == 1  and np.mod(self.grid_pos[index],4)<3: # move up
                    
                    new_pos = self.grid_pos[index]+1
                    self, delete_vector[index] = self.checkMovement(index,new_pos)
                
            if not(delete_vector[index]):
                self.drawTile(index)
        for delete_tile in delete_vector:
            if delete_tile:
                self.number = np.delete(self.number,delete_tile)
                self.grid_pos = np.delete(self.grid_pos,delete_tile)
                print('deleted tile')
        return self
    def drawTile(self,index):
        current_number = self.number[index]
        current_x = np.mod(self.grid_pos[index],4).astype(int)
        current_y = np.floor(self.grid_pos[index]/4).astype(int)
        screen.blit(self.icons[current_number],(current_y*self.icons[current_number].get_width(),
                                      current_x*self.icons[current_number].get_height()))
    def checkMovement(self,index,new_pos):
        delete_tile = False
        if sum(np.isin(self.grid_pos,new_pos))==0:
            self.grid_pos[index] = new_pos                        
        else:
            new_pos_index_logical = self.grid_pos == new_pos
            new_pos_number = self.number[new_pos_index_logical]
            if self.number[index] + new_pos_number == 3:
                print('tile 1 = ' + str(self.number[index]) + ' tile 2 = ' + str(new_pos_number))
                self.number[new_pos_index_logical] = 3
                delete_tile = True
                
                
        return self, delete_tile
        
        
   
    

In [132]:
running = True
current_x = 0
current_y = 0
screen = pg.display.set_mode((800,600))
screen.fill((0,0,255))
pg.display.set_caption("Threes!")


grid  =pg.image.load('C:/Henry/PythonProjects/Pygame/Threes/images/grid.png')
tile_list = [1,2,3,6,12,24,48,96,192,384,768,1536,3072,6144]
start_num = 2

game_tiles = Tiles(start_num)

ex_state=True

while running:
    #pg.time.delay(100)
    screen.fill((0,0,255))
    screen.blit(grid,(0,0))
    keys = pg.key.get_pressed()
    if ex_state and sum(keys)>0:
        current_move, move_order = game_tiles.determineMove(keys)
        
        ex_state = False
    else:
        current_move = 0
        move_order = np.argsort(game_tiles.grid_pos)
        move_order = np.argsort(game_tiles.grid_pos)
    game_tiles = game_tiles.updatePosition(current_move,move_order)
        
    
            
    pg.display.update()

    
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
            pg.display.quit()
        if event.type == pg.MOUSEBUTTONDOWN:
            print('mouse clicked')
        if event.type == pg.KEYUP:
            ex_state = True;

    
      
            

tile 1 = 2 tile 2 = [1]
deleted tile


C:\Users\hjali\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: in the future insert will treat boolean arrays and array-likes as boolean index instead of casting it to integer
C:\Users\hjali\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: FutureWarning: in the future insert will treat boolean arrays and array-likes as boolean index instead of casting it to integer


In [117]:

pg.display.quit()


In [101]:
a = np.ones((2,),dtype = bool)
sum(a)

2

In [43]:
a = False
if not(a):
    print('False')

False


In [25]:
a = np.array([0, 1, 2])
np.where(a < 4, a, -1)  # -1 is broadcast


array([0, 1, 2])

In [23]:
a.shape

(3, 3)